In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-learn==0.22.1
!pip install missingpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-learn: filename=scikit_learn-0.22.1-cp310-cp310-linux_x86_64.whl size=20732634 sha256=8cb63a1bc69788f3f789e58d3c154b8e56b455f0f1aa85f2023a7f469a8a8ec3
  Stored in directory: /root/.cache/pip/wheels/ca/81/17/bedd2679672da7fb35b1984624fa1513d04e026878d72d6a42
Successfully built scikit-learn
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.10.1 requires scikit-learn>=1.0.2, but you have scikit-learn 0.22.1 which is incompatible.
mlxtend 0.22.0 requires scikit-learn>=1.0.2, but you have scikit-learn 0.22.1 which is incompatible.
s

In [ ]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
# from missingpy import MissForest
import warnings
warnings.filterwarnings("ignore")

평 = m^2 / 3.3058

In [ ]:
path = '/content/drive/MyDrive/DACON/electronic/'

In [ ]:
train = pd.read_csv(path+'train.csv')
info = pd.read_csv(path+'building_info.csv')
test = pd.read_csv(path+'test.csv')
submission = pd.read_csv(path+'sample_submission.csv')

In [ ]:
# prepare data features
def __date_prep(df):

    df['datetime'] = pd.to_datetime(df['datetime'])
    df['hour'] = df['datetime'].dt.hour
    df['dow'] = df['datetime'].dt.weekday
    df['date'] = df['datetime'].dt.date.astype('str')
    df['day'] = df['datetime'].dt.day
    df['month'] = df['datetime'].dt.month
    df['sin_time'] = np.sin(2*np.pi*df.hour/24)
    df['cos_time'] = np.cos(2*np.pi*df.hour/24)
    df['THI'] = 9/5*df['temperature'] - 0.55*(1-df['humidity']/100)*(9/5*df['humidity']-26)+32
    df['WC'] = 13.12+0.6215*df['temperature']-13.947*df['windspeed']**0.16+0.486*df['temperature']*df['windspeed']**0.16
    df['Hindex'] = df['temperature'] + (0.5555 * math.exp(-10))
    # df['work_hour'] = ((df['hour']>=8) & (df['hour']<=19)).astype(int)
#     df['lunch_hour'] = ((df['hour']>=11) & (df['hour']<=13) & (df['dow']<=4)).astype(int)
#     df['lunch_hour2'] = ((df['hour']>=12) & (df['hour']<=14) & (df['dow']>4)).astype(int)

#     df['dinner_hour'] = ((df['hour']>=17) & (df['hour']<=22)).astype(int)
#     df['dinner_hour2'] = ((df['hour']>=18) & (df['dow']>=4) & (df['dow']<=5)).astype(int)

    cdhs = np.array([])
    for num in range(1,101,1):
        temp = df[df['num'] == num]
        cdh = CDH(temp['temperature'].values)
        cdhs = np.concatenate([cdhs, cdh])
    df['CDH'] = cdhs

    # FEATURE: saturday, sunday and speical holidays flagged as `holiday` flag
    special_days = ['2020-01-01', '2020-06-06', '2020-08-15', '2020-08-17', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-03-01', '2020-04-15', '2020-04-30', '2020-05-05', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                    '2021-01-01', '2021-02-11', '2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                    '2022-01-01', '2022-01-31', '2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09',  '2022-10-10', '2022-12-25']
    df['holiday'] = df['dow'].isin([5,6]).astype(int)
    df.loc[df.date.isin(special_days), 'holiday'] = 1

    # FEATURE: `hot` flag when the next day is holiday
    hot = df.groupby('date').first()['holiday'].shift(-1).fillna(0).astype(int)
    hot = hot.to_frame().reset_index().rename({'holiday': "hot"}, axis=1)
    df = df.merge(hot, on='date', how='left')

    # FEATURE: `cumhol` - how many days left in 연휴
    h = (df.groupby('date').first()['holiday'] != 0).iloc[::-1]
    df1 = h.cumsum() - h.cumsum().where(~h).ffill().fillna(0).astype(int).iloc[::-1]
    df1 = df1.to_frame().reset_index().rename({'holiday': "cumhol"}, axis=1)
    df = df.merge(df1, on='date', how='left')

    return df

In [ ]:
# interpolate NA values in test dataset
def interpolate_(train_df):
    # https://dacon.io/competitions/official/235736/codeshare/2844?page=1&dtype=recent
    # 에서 제안된 방법으로
    __methods = {
        'precipitation': 'quadratic',
        'windspeed':'quadratic',
        'humidity':'quadratic',
        # 'insolation': 'pad'
    }

    for col, method in __methods.items():
        train_df[col] = train_df[col].interpolate(method=method)
        if method == 'quadratic':
            train_df[col] = train_df[col].interpolate(method='linear')

def interpolate2_(train_df):
    # https://dacon.io/competitions/official/235736/codeshare/2844?page=1&dtype=recent
    # 에서 제안된 방법으로
    __methods = {
        'precipitation': 'linear',
        # 'windspeed':'quadratic',
        # 'humidity':'quadratic',
        # 'insolation': 'pad'
    }

    for col, method in __methods.items():
        train_df[col] = train_df[col].interpolate(method=method)
        if method == 'quadratic':
            train_df[col] = train_df[col].interpolate(method='linear')

In [ ]:
info

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,0,0,0
1,2,건물기타,122233.47,99000.00,0,0,0
2,3,건물기타,171243.00,113950.00,40,0,0
3,4,건물기타,74312.98,34419.62,60,0,0
4,5,건물기타,205884.00,150000.00,0,2557,1000
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,0,0,0
96,97,호텔및리조트,55144.67,25880.00,0,0,0
97,98,호텔및리조트,53578.62,17373.75,0,0,0
98,99,호텔및리조트,53499.00,40636.00,0,0,0


In [ ]:
info['태양광용량(kW)'] = info['태양광용량(kW)'].replace('-', 0)
info['ESS저장용량(kWh)'] = info['ESS저장용량(kWh)'].replace('-', 0)
info['PCS용량(kW)'] = info['PCS용량(kW)'].replace('-', 0)
# info.drop(columns=['건물유형'], inplace=True)

In [ ]:
def data_processing(df, info):
    df = pd.merge(df, info, how='inner', on='건물번호')
    df.drop(columns=['num_date_time'], inplace=True)
    return df

In [ ]:
train

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


In [ ]:
train = data_processing(train, info)
test = data_processing(test, info)
train.drop(columns=['일조(hr)', '일사(MJ/m2)'], inplace=True)

In [ ]:
train.columns = ['num', 'datetime', 'temperature', 'precipitation', 'windspeed', 'humidity', 'target', 'floor_area', 'cooling_area', 'solar', 'ESS', 'PCS']
test.columns = ['num', 'datetime', 'temperature', 'precipitation', 'windspeed', 'humidity', 'floor_area', 'cooling_area', 'solar', 'ESS', 'PCS']

In [ ]:
train.isnull().sum()

num                   0
datetime              0
temperature           0
precipitation    160069
windspeed            19
humidity              9
target                0
floor_area            0
cooling_area          0
solar                 0
ESS                   0
PCS                   0
dtype: int64

In [ ]:
test.isnull().sum()

num              0
datetime         0
temperature      0
precipitation    0
windspeed        0
humidity         0
floor_area       0
cooling_area     0
solar            0
ESS              0
PCS              0
dtype: int64

In [ ]:
train.to_csv(path + 'train_process.csv', index=False)
test.to_csv(path + 'test_process.csv', index=False)

In [ ]:
test.columns

Index(['건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '연면적(m2)',
       '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)'],
      dtype='object')

In [ ]:
train.columns

Index(['건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '전력소비량(kWh)',
       '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)'],
      dtype='object')

In [ ]:
train.columns

Index(['건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', '전력소비량(kWh)', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)',
       'ESS저장용량(kWh)', 'PCS용량(kW)'],
      dtype='object')

In [ ]:
train.columns = []

In [ ]:
train.columns

Index(['num_date_time', '건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)',
       '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)',
       'ESS저장용량(kWh)', 'PCS용량(kW)'],
      dtype='object')

In [ ]:
train.isnull().sum()

num_date_time         0
건물번호                  0
일시                    0
기온(C)                 0
강수량(mm)          160069
풍속(m/s)              19
습도(%)                 9
일조(hr)            75182
일사(MJ/m2)         87913
전력소비량(kWh)            0
건물유형                  0
연면적(m2)               0
냉방면적(m2)              0
태양광용량(kW)             0
ESS저장용량(kWh)          0
PCS용량(kW)             0
dtype: int64

In [ ]:
null_columns = ['강수량(mm)', '일조(hr)', '일사(MJ/m2)']

In [ ]:
input_columns = list(set(train.columns) - set(['전력소비량(kWh)', '강수량(mm)', '일조(hr)', '일사(MJ/m2)']))
imputer = datawig.SimpleImputer(input_columns=input_columns,
                                output_column='풍속(m/s)')
imputer.fit(train_df=train, num_epochs=50)
null_train = train[train['풍속(m/s)'].isnull()]
null_imputed = imputer.predict(null_train)
imputed_train = pd.DataFrame(null_imputed)

In [ ]:
train.isnull().sum(

)

num_date_time         0
건물번호                  0
일시                    0
기온(C)                 0
강수량(mm)          160069
풍속(m/s)              19
습도(%)                 9
일조(hr)            75182
일사(MJ/m2)         87913
전력소비량(kWh)            0
건물유형                  0
연면적(m2)               0
냉방면적(m2)              0
태양광용량(kW)             0
ESS저장용량(kWh)          0
PCS용량(kW)             0
dtype: int64